In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine('sqlite:///../Resources/hawaii.sqlite')

In [ ]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
print(session.query(measurement.date).order_by(measurement.date.desc()).first())


In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
last_date = dt.date(2017, 8, 23)

# Calculate the date one year from the last date in data set.
one_year_ago = last_date - dt.timedelta(days= 365)

# Perform a query to retrieve the data and precipitation scores
last_year_query = session.query(measurement.date, measurement.prcp).filter(measurement.date >= one_year_ago).all()


# Save the query results as a Pandas DataFrame. Explicitly set the column names
df = pd.DataFrame(last_year_query, columns=['Date','Precipitation (inches)'])


# Sort the dataframe by date
df = df.sort_values('Date')
df.set_index('Date', inplace= True)


# Use Pandas Plotting with Matplotlib to plot the data
df.plot(rot= 90)

plt.ylabel('Precipitation (inches)')
plt.xlabel('Date')
plt.legend(loc= 'upper right', fontsize = 12 )
plt.show()

In [ ]:
# Use Pandas to calculate the summary statistics for the precipitation data
df.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number of stations in the dataset
stations = session.query(measurement).group_by(measurement.station).count()
stations

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?)
# List the stations and their counts in descending order.
active_stations = session.query(measurement.station, func.count(measurement.station)).\
    group_by(measurement.station).order_by(func.count(measurement.station).desc()).all()
active_stations


In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
low = session.query(func.min(measurement.tobs)).filter(measurement.station == 'USC00519281').all()
high = session.query(func.max(measurement.tobs)).filter(measurement.station == 'USC00519281').all()
avg = session.query(func.avg(measurement.tobs)).filter(measurement.station == 'USC00519281').all()
print(low, high, avg)


In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
# Find the last day that observation data was recorded

# print(session.query(measurement.date).order_by(measurement.date.desc()).filter(measurement.station == 'USC00519281').first())

# After determining the last day was August 18th, 2017, get the last 12 months of temperature observation data for this station

last_date_519281 = dt.date(2017,8,18)
past_year_519281 = last_date_519281 - dt.timedelta(days=365)
past_year_query_519281 = session.query(measurement.date, measurement.tobs).\
    filter(measurement.date >= past_year_519281, measurement.station == 'USC00519281').all()

df_519281 = pd.DataFrame(past_year_query_519281)
df_519281.plot.hist(bins= 12)
plt.xlabel('Temperature')
plt.show()


# Close Session

In [ ]:
# Close Session
session.close()